In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime
import regex as re
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from xgboost import XGBClassifier
import pandas as pd


In [2]:
imonitor = pd.read_csv('data/imonitor_1703.csv')
imonitor.head()

C:\Users\mogam\AppData\Local\Temp\ipykernel_768\2747984450.py:1: DtypeWarning: Columns (1,4,11,15,24,25,42,56,62,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84) have mixed types. Specify dtype option on import or set low_memory=False.
  imonitor = pd.read_csv('data/imonitor_1703.csv')


,Survey ID,Created Date,Facility name and MFL Code if applicable,Facility ownership,Please specify,County,What is your month; and year of birth,How do you consider yourself?,What is the highest level of education you completed?,Please specify.1,...,how long do you wait on average to get a service; which service was that?,Do you consider the waiting time for lab test results long?,how long do you wait on average to get your lab test result?,Does the facility offer support groups?,Specify the support group you belong to,In your opinion are the services offered at this facility youth friendly?,What measures have been put in place to create GBV awareness and its harmful effects within the community?,Please Specify,PWD In your opinion are the services offered at this facility persons-with-disability friendly?,What are the top 1-3 things you don’t like about this facility with regards to care and treatment?
0,2390063,04-Dec-23,BABA DOGO HEALTH CENTRE,GOK,NaN,Nairobi,1977-09-03,Male,Primary school,NaN,...,NaN,No,NaN,Yes,Adults,Yes,Presence of GBV Desk;,Chiefs office,Yes,NaN
1,2390062,04-Dec-23,BABA DOGO HEALTH CENTRE,GOK,NaN,Nairobi,1972-08-12,Female,Secondary school,NaN,...,NaN,No,NaN,No,NaN,Yes,Presence of GBV Desk;,Chiefs office,NaN,NaN
2,2390061,04-Dec-23,BABA DOGO HEALTH CENTRE,GOK,NaN,Nairobi,1984-08-31,Female,Primary school,NaN,...,NaN,Yes,2 hours,No,NaN,Yes,Presence of GBV Desk;,Chiefs office,Yes,NaN
3,2390060,04-Dec-23,BABA DOGO HEALTH CENTRE,GOK,NaN,Nairobi,1977-05-07,Female,Primary school,NaN,...,NaN,No,NaN,No,NaN,Yes,Presence of GBV Desk;,Police station,NaN,NaN
4,2390059,04-Dec-23,BABA DOGO HEALTH CENTRE,GOK,NaN,Nairobi,1987-06-13,Male,Vocational training or technician,NaN,...,1 hour,Yes,2 hours,Yes,Adults,Yes,Presence of GBV Desk;,Police station,NaN,NaN


In [3]:
imonitor.shape

(46549, 85)

In [4]:
# Find and drop columns that contain "Please specify" or "Please Specify"
cols_to_drop = [col for col in imonitor.columns if "Please specify" in col or "Please Specify" in col]

# Drop these columns from the DataFrame in a single operation
imonitor.drop(cols_to_drop, axis=1, inplace=True)

In [5]:
imonitor.shape

(46549, 69)

In [6]:
column_name_mapping = {
    "Survey ID": "SurveyID",
    "Organization name coordinating the feedback from the clients": "OrgFeedbackCoordinator",
    "Created Date": "FeedbackDate",
    "Facility name and MFL Code if applicable": "FacilityName",
    "Facility ownership": "FacilityOwnership",
    "County": "FacilityCounty",
    "What is your month; and year of birth": "BirthMonthYear",
    "How do you consider yourself? ": "SelfIdentity",
    "What is the highest level of education you completed?": "EducationLevel",
    "What is your current marital status? ": "MaritalStatus",
    "Which county do you currently live in?": "ResidenceCounty",
    "What are your sources of income?": "IncomeSources",
    "For how long have you been accessing services (based on the expected package of services) in this facility?": "ServiceAccessDuration",
    "Are you aware of the package of services that you are entitled to?": "ServicesAwareness",
    "According to you; which HIV related services are you likely to receive in this facility?": "ExpectedHIVServices",
    "Is there a service that you needed that was not provided?": "UnprovidedService",
    "Facility name no service": "UnprovidedServiceFacilityName",
    "For that service that was not provided; were you referred?": "ReferralForUnprovidedService",
    "If referred; did you receive the service where you were referred to?": "ReferralServiceReceived",
    "If Yes which Service/Test/Medicine ": "ReceivedServiceDetail",
    "On a scale of 1 to 5; how satisfied are you with the package of services received in this facility? If 1 is VERY UNSATISFIED and 5 is VERY SATISFIED.": "ServiceSatisfaction",
    "What did you like about the services you received?": "ServicesLiked",
    "What did you not like about the services you received?": "ServicesDisliked",
    "In your opinion what would you like to be improved?": "ImprovementSuggestions",
    "Do you face any challenges when accessing the services at the facility?": "AccessChallenges",
    "Common issues that can be added in the drop-down box": "CommonIssuesDropdown",
    "In your opinion what can be done to improve access to the services you seek at the facility?": "AccessImprovementSuggestions",
    "Was confidentiality considered while you were being served?": "Confidentiality",
    "Are there age-appropriate health services for specific groups?": "AgeAppropriateServices",
    "Does the facility allow you to share your concerns with the administration?": "ConcernsSharing",
    "Do you know your health-related rights as a client of this facility?": "RightsAwareness",
    "Have you ever been denied services at this facility?": "ServiceDenial",
    "Facility name denied service": "ServiceDenialFacilityName",
    "Why": "ServiceDenialSpecify",
    "Are you comfortable with getting services at this facility": "ComfortWithServices",
    "Have you ever been counseled? ": "CounselingReceived",
    "Did you identify any gaps in the facility when you tried to access the services": "IdentifiedGaps",
    "Service type": "ServiceGapsType",
    "Are the HIV testing services readily available when required? ": "HIVTestingAvailability",
    "Have you ever Interrupted your treatment?": "TreatmentInterruption",
    "Are the PMTCT services readily available when required?": "PMTCTServiceAvailability",
    "Were reasons provided as to why these services were not available?": "PMTCTServiceNonavailabilityReasons",
    "Are the HIV prevention; testing; treatment and care services adequate for KPs? ": "KPServiceAdequacy",
    "Were reasons provided as to why these services were not available?.1": "KPServiceNonavailabilityReasons",
    "What are the barriers to uptake of VMMC by males 25+years and above?": "VMMCBarriers",
    "What are some of the current site level practices that community members like and would love to maintain for KP/PP ?": "KPCommunityPreferredPractices",
    "What would you like this facility to change/do better?": "ChangeSuggestions",
    "Throughout your visit what did you find interesting/pleasing about this facility that should be emulated by other facilities?": "PositiveObservations",
    "What do you think can be improved": "GeneralImprovementSuggestions",
    "Anything else that you would like to mention?": "AdditionalComments",
    "What are the top 1-3 things you like about this facility with regards to care and treatment? ": "TopFacilityFeatures",
    "Facility Level": "FacilityLevel",
    "Facility Operation times": "OperationTimes",
    "Facility Operation Days ": "OperationDays",
    "What are your preferred days of visiting the facility": "PreferredVisitDays",
    "What are your preferred time of visiting the facility": "PreferredVisitTimes",
    "According to you, which HIV related service/tests/medicine are you likely to receive in this facility?": "ExpectedHIVServices2",
    "Is there a service/test/medicine that you needed that was not provided?": "UnprovidedService2",
    "which service/test/medicine?": "UnprovidedServiceDetail",
    "On a scale of 1-5; how clean do you find the facility?": "FacilityCleanliness",
    "How do you reach this facility?": "FacilityAccessMode",
    "How long does it take to reach this facility?": "FacilityAccessTime",
    "On a scale of 1-5; how accessible do you find this facility?": "FacilityAccessibility",
    "Do you consider the waiting time to be seen at this facility long?": "WaitingTimeOpinion",
    "how long do you wait on average to get a service; which service was that?": "AverageWaitingTime",
    "Do you consider the waiting time for lab test results long?": "LabResultsWaitingTimeOpinion",
    "how long do you wait on average to get your lab test result?": "AverageLabResultsWaitingTime",
    "Does the facility offer support groups?": "SupportGroupAvailability",
    "Specify the support group you belong to": "SpecifySupportGroup",
    "In your opinion are the services offered at this facility youth friendly?": "YouthFriendlyServices",
    "What measures have been put in place to create GBV awareness and its harmful effects within the community? ": "GBVAwarenessMeasures",
    "PWD In your opinion are the services offered at this facility persons-with-disability friendly?": "PWDFriendlyServicesOpinion",
    "What are the top 1-3 things you don’t like about this facility with regards to care and treatment?": "TopFacilityDislikes"
}

df = imonitor.rename(columns=column_name_mapping)

In [7]:
for column in df.columns:
    print(column)

SurveyID
FeedbackDate
FacilityName
FacilityOwnership
FacilityCounty
BirthMonthYear
SelfIdentity
EducationLevel
MaritalStatus
ResidenceCounty
IncomeSources
ServiceAccessDuration
ServicesAwareness
ExpectedHIVServices
UnprovidedService
Facility name
ReferralForUnprovidedService
ReferralServiceReceived
ReceivedServiceDetail
ServiceSatisfaction
ServicesLiked
ServicesDisliked
ImprovementSuggestions
AccessChallenges
CommonIssuesDropdown
AccessImprovementSuggestions
Confidentiality
AgeAppropriateServices
ConcernsSharing
RightsAwareness
ServiceDenial
ServiceDenialFacilityName
ServiceDenialSpecify
ComfortWithServices
CounselingReceived
IdentifiedGaps
ServiceGapsType
HIVTestingAvailability
TreatmentInterruption
PMTCTServiceAvailability
PMTCTServiceNonavailabilityReasons
KPServiceAdequacy
KPServiceNonavailabilityReasons
VMMCBarriers
KPCommunityPreferredPractices
ChangeSuggestions
PositiveObservations
GeneralImprovementSuggestions
AdditionalComments
TopFacilityFeatures
FacilityLevel
OperationTimes


In [8]:
columns_to_clean1 = [
    'WaitingTimeOpinion',
    'LabResultsWaitingTimeOpinion'
]

def replace_dont_know(df, column):
    df[column] = df[column].replace("Dont Know", "I don't know", regex=False)
    return df

for column in columns_to_clean1:
    df = replace_dont_know(df, column)

In [9]:
columns_to_clean2 = [
    'FacilityCleanliness',
    'FacilityAccessibility'
    ]

def replace_mixed_with_text(df, column_name):
    def replace_value(value):
        satisfaction_map = {
            1: 'Very Unsatisfied',
            2: 'Unsatisfied',
            3: 'Okay',
            4: 'Satisfied',
            5: 'Very Satisfied'
        }
        if isinstance(value, str) and value[0].isdigit():
            num = int(value[0])
        elif isinstance(value, int):
            num = value
        else:
            return value

        return satisfaction_map.get(num, value)

    df[column_name] = df[column_name].apply(replace_value)
    return df

for column in columns_to_clean2:
    df = replace_mixed_with_text(df, column)

In [10]:
def standardize_satisfaction(df, column_name):
    # Mapping for consolidating variations of satisfaction levels
    satisfaction_map = {
        '5': 'Very Satisfied',
        5.0: 'Very Satisfied',
        '4': 'Satisfied',
        4.0: 'Satisfied',
        '3': 'Okay',
        3.0: 'Okay',
        '2': 'Unsatisfied',
        2.0: 'Unsatisfied',
        '1': 'Very Unsatisfied',
        1.0: 'Very Unsatisfied',
        'Dissatisfied': 'Unsatisfied'
    }
    
    # Replace values based on the map
    df[column_name] = df[column_name].replace(satisfaction_map)
    return df

df = standardize_satisfaction(df, 'ServiceSatisfaction')


In [11]:
print(df['FacilityLevel'].value_counts())

FacilityLevel
4.0    4802
3.0    4515
2.0    2889
5.0    2240
1.0     556
6.0      14
Name: count, dtype: int64


In [12]:
def standardize_facility(df, column_name):
    # Mapping for consolidating variations of satisfaction levels
    satisfaction_map = {
        1.0: 'Community Health Unit',
        2.0: 'Dispensaries and Private Clinics',
        3.0: 'Health Centers',
        4.0: 'Sub-County Hospitals',
        5.0: 'County Referral Hospitals',
        6.0: 'National Referral Hospitals',
    }
    
    # Replace values based on the map
    df[column_name] = df[column_name].replace(satisfaction_map)
    return df

df = standardize_facility(df, 'FacilityLevel')

In [13]:
def replace_symbols_and_words(df, column_name):
    df[column_name] = df[column_name].str.replace('<', 'Less than', regex=False)
    df[column_name] = df[column_name].str.replace('>', 'More than', regex=False)
    df[column_name] = df[column_name].str.replace('minutes', 'mins', regex=False)
    return df

df = replace_symbols_and_words(df, 'FacilityAccessTime')

In [14]:
def replace_symbols_and_words(df, column_name):
    df[column_name] = df[column_name].str.replace('Less than 30mins', 'Less than 30 mins', regex=False)
    df[column_name] = df[column_name].str.replace('More than45 mins', 'More than 45 mins', regex=False)
    return df

df = replace_symbols_and_words(df, 'FacilityAccessTime')

In [15]:
def remove_trailing_semicolons(df, column_names):
    for column in column_names:
        if column in df.columns:
            df[column] = df[column].str.rstrip(';')
    return df

df = pd.DataFrame(df)

columns_to_clean = ['ExpectedHIVServices', 'OperationTimes', 'OperationDays', 'PreferredVisitDays', 'PreferredVisitTimes', 'GBVAwarenessMeasures']

df2 = remove_trailing_semicolons(df, columns_to_clean)

In [16]:
def calculate_age(birth_date_str):
    if isinstance(birth_date_str, str):
        # Parse the birth date string into a datetime object
        birth_date = datetime.strptime(birth_date_str, "%Y-%m-%d")
        
        # Get today's date
        today = datetime.today()
        
        # Calculate age
        age = today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))
        
        return age
    else:
        # Return None or an appropriate value for missing or invalid input
        return None
    
df2['Age'] = df2['BirthMonthYear'].apply(calculate_age)

In [17]:
def remove_invalid_age_rows(df2, age_column):
    valid_age_condition = (df[age_column] >= 0) & (df[age_column] <= 100)
    
    # Keep only rows with valid age
    cleaned_df = df2[valid_age_condition].copy()
    
    return cleaned_df
df3 = remove_invalid_age_rows(df2, 'Age')

In [18]:
df3.drop(columns=['BirthMonthYear'], inplace=True)

In [19]:
def convert_mixed_dates(date_column):
    """
    This function takes a Pandas Series of mixed dates and Excel serial dates and converts them to datetime objects.
    
    Parameters:
    date_column (pd.Series): A pandas Series with mixed date formats and serial dates.
    
    Returns:
    pd.Series: A pandas Series with all dates converted to datetime objects.
    """
    # Define the epoch start for Excel's serial date format
    excel_epoch = pd.Timestamp('1899-12-30')
    converted_dates = []

    for date in date_column:
        if isinstance(date, str) and re.match(r'^\d+(\.\d+)?$', date):
            # If it's a string that looks like a serial date, convert it
            serial_value = float(date)
            converted_date = excel_epoch + pd.to_timedelta(serial_value, unit='D')
        elif isinstance(date, (int, float)):
            # If it's a numeric type, assume it's a serial date
            converted_date = excel_epoch + pd.to_timedelta(date, unit='D')
        else:
            # Otherwise, try to parse it as a regular date
            converted_date = pd.to_datetime(date, errors='coerce')

        # Append the result, which will be NaT if parsing failed
        converted_dates.append(converted_date)

    return pd.Series(converted_dates)

convert_mixed_dates(df3['FeedbackDate'])


0       2023-12-04
1       2023-12-04
2       2023-12-04
3       2023-12-04
4       2023-12-04
           ...    
41839   2022-10-04
41840   2022-10-03
41841   2022-10-04
41842   2022-10-03
41843   2022-09-28
Length: 41844, dtype: datetime64[ns]

In [20]:
df3.to_csv('data/cleaned.csv', index=False)

In [21]:
df['FacilityCounty'].value_counts()

FacilityCounty
Homabay    16563
Kisumu      6267
Kilifi      6228
Mombasa     4540
Nairobi     4170
Makueni     3888
Narok        471
Samburu       81
Turkana       15
Nakuru        13
Name: count, dtype: int64

In [22]:
columns_to_keep = [
    'SurveyID', 'FeedbackDate', 'FacilityCounty', 'FacilityLevel', 'FacilityOwnership', 'ServicesLiked', 'ServicesDisliked', 'ImprovementSuggestions',
    'AccessImprovementSuggestions', 'PositiveObservations',
    'GeneralImprovementSuggestions', 'AdditionalComments', 'TopFacilityFeatures', 'ServiceSatisfaction', 'TopFacilityDislikes'
]

df_selected = df3[columns_to_keep]

df_selected.to_csv('data/clm_open_ended.csv', index=False)

In [23]:
missing_percentage = df3.isnull().mean() * 100

threshold = 60

columns_to_drop = missing_percentage[missing_percentage > threshold].index.tolist()

print("Columns to drop:", columns_to_drop)

print("Number of columns to drop:", len(columns_to_drop))

df3.drop(columns=columns_to_drop, axis=1, inplace=True)

print("DataFrame shape after dropping columns:", df3.shape)

Columns to drop: ['Facility name', 'ReferralForUnprovidedService', 'ReferralServiceReceived', 'ReceivedServiceDetail', 'CommonIssuesDropdown', 'ServiceDenialFacilityName', 'ServiceDenialSpecify', 'ServiceGapsType', 'HIVTestingAvailability', 'TreatmentInterruption', 'PMTCTServiceAvailability', 'PMTCTServiceNonavailabilityReasons', 'KPServiceAdequacy', 'KPServiceNonavailabilityReasons', 'VMMCBarriers', 'KPCommunityPreferredPractices', 'ChangeSuggestions', 'TopFacilityFeatures', 'FacilityLevel', 'OperationTimes', 'OperationDays', 'PreferredVisitDays', 'PreferredVisitTimes', 'FacilityCleanliness', 'FacilityAccessMode', 'FacilityAccessTime', 'FacilityAccessibility', 'WaitingTimeOpinion', 'AverageWaitingTime', 'LabResultsWaitingTimeOpinion', 'AverageLabResultsWaitingTime', 'SupportGroupAvailability', 'SpecifySupportGroup', 'YouthFriendlyServices', 'GBVAwarenessMeasures', 'PWDFriendlyServicesOpinion', 'TopFacilityDislikes']
Number of columns to drop: 37
DataFrame shape after dropping columns:

In [24]:
threshold_percentage = 100

threshold = len(df3.columns) * (threshold_percentage / 100)

data = df3.dropna(thresh=threshold).copy()

print("Original DataFrame shape:", df3.shape)
print("Cleaned DataFrame shape:", data.shape)

rows_dropped = df3.shape[0] - data.shape[0]
print("Rows dropped:", rows_dropped)

Original DataFrame shape: (41844, 32)
Cleaned DataFrame shape: (26214, 32)
Rows dropped: 15630


In [25]:
# General descriptive statistics
data.describe()

,SurveyID,Age
count,2.621400e+04,26214.000000
mean,8.683058e+05,41.976387
std,1.002974e+06,12.763959
min,1.796510e+05,0.000000
25%,1.915855e+05,34.000000
50%,2.026525e+05,41.000000
75%,2.338173e+06,50.000000
max,2.390027e+06,100.000000


In [26]:
# Descriptive statistics for categorical data
data.describe(include=['object'])

,FeedbackDate,FacilityName,FacilityOwnership,FacilityCounty,SelfIdentity,EducationLevel,MaritalStatus,ResidenceCounty,IncomeSources,ServiceAccessDuration,...,AgeAppropriateServices,ConcernsSharing,RightsAwareness,ServiceDenial,ComfortWithServices,CounselingReceived,IdentifiedGaps,PositiveObservations,GeneralImprovementSuggestions,AdditionalComments
count,26214,26214,26214,26214,26214,26214,26214,26214,26214,26214,...,26214,26214,26214,26214,26214,26214,26214,26214,26214,26214
unique,9987,1254,6,10,7,8,9,12,27,4,...,2,2,2,2,2,3,2,9082,7579,7946
top,09-Oct-23,Malindi Sub-County Hospital,GOK,Homabay,Female,Primary school,Married,Homabay,Business;,More than one year,...,Yes,Yes,Yes,No,Yes,Yes,No,Good services,No comment,No comment
freq,1970,835,21836,9558,18023,13216,15742,9349,6057,23530,...,23882,25604,24062,26142,25936,22612,25038,734,2221,3433


In [27]:
data['ServiceSatisfaction'].value_counts()

ServiceSatisfaction
Satisfied                14620
Very Satisfied           11040
Okay                       228
Unsatisfied                222
Do not know                 47
Very Unsatisfied            36
Prefer not to answer        21
Name: count, dtype: int64

In [28]:
recategorization_mapping = {
    'Very Satisfied': 2,
    'Satisfied': 1,
    'Okay': 1,
    'Unsatisfied': 0,
    'Very Unsatisfied': 0,
    #'Unknown': 0,
    'Do not know': 99,
    'Prefer not to answer ': 99
}

data.loc[:, 'ServiceSatisfaction'] = data['ServiceSatisfaction'].replace(recategorization_mapping)

# After replacement, you might want to ensure the data type is what you expect
# For example, if you want to ensure it's an integer (especially if NaN values are not expected)
data['ServiceSatisfaction'] = data['ServiceSatisfaction'].astype(int)

# Verify the changes
print(data['ServiceSatisfaction'].value_counts())

ServiceSatisfaction
1     14848
2     11040
0       258
99       68
Name: count, dtype: int64


C:\Users\mogam\AppData\Local\Temp\ipykernel_768\1833295796.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.loc[:, 'ServiceSatisfaction'] = data['ServiceSatisfaction'].replace(recategorization_mapping)


In [29]:
model_data = data[data.ServiceSatisfaction != 99]

In [30]:
model_data = model_data.drop(['SurveyID', 'FeedbackDate', 'FacilityName', 'ResidenceCounty', 'ServicesLiked', 'ServicesDisliked', 'ImprovementSuggestions', 'AccessImprovementSuggestions', 'PositiveObservations', 'GeneralImprovementSuggestions', 'AdditionalComments', 'Age'], axis=1)

In [31]:
model_data['FacilityCounty'].value_counts()

FacilityCounty
Homabay    9496
Kilifi     5431
Mombasa    3823
Makueni    3075
Kisumu     2878
Nairobi    1290
Narok       126
Samburu      19
Turkana       7
Nakuru        1
Name: count, dtype: int64

In [32]:
# Assuming you have a pandas DataFrame called df
subset_df = model_data[model_data['FacilityCounty'].isin(['Nairobi', 'Kisumu'])]

In [33]:
subset_df['FacilityCounty'].value_counts()

FacilityCounty
Kisumu     2878
Nairobi    1290
Name: count, dtype: int64

In [34]:
subset_df['ServiceSatisfaction'].value_counts()

ServiceSatisfaction
1    2036
2    2013
0     119
Name: count, dtype: int64

In [35]:
# Assuming subset_df is your DataFrame and 'ServiceSatisfaction' is the column of interest

# Split the dataset into separate groups based on 'ServiceSatisfaction'
class_3_df = subset_df[subset_df['ServiceSatisfaction'] == 2]
class_2_df = subset_df[subset_df['ServiceSatisfaction'] == 1]
class_1_df = subset_df[subset_df['ServiceSatisfaction'] == 0]

# Get the target number of instances to match, which is the number of instances in class 1
target_number = class_1_df.shape[0]

# Randomly sample from classes 3 and 2 to match the number of instances in class 1
class_3_sampled_df = class_3_df.sample(n=target_number, random_state=42)
class_2_sampled_df = class_2_df.sample(n=target_number, random_state=42)

balanced_df = pd.concat([class_3_sampled_df, class_2_sampled_df, class_1_df])

In [36]:
balanced_df['FacilityCounty'].value_counts()

FacilityCounty
Kisumu     268
Nairobi     89
Name: count, dtype: int64

In [37]:
balanced_df['ServiceSatisfaction'].value_counts()

ServiceSatisfaction
2    119
1    119
0    119
Name: count, dtype: int64

In [38]:
ordinal_vars = balanced_df['ServiceSatisfaction']
nominal_vars = [col for col in balanced_df.columns if balanced_df[col].dtype == 'object' and col not in ordinal_vars]
encoded_data = pd.get_dummies(balanced_df, columns=nominal_vars)

# This automatically drops the original nominal columns and adds the one-hot encoded columns
print("NaN counts after pandas get_dummies:", encoded_data.isnull().sum().sum())

NaN counts after pandas get_dummies: 0


In [39]:
X = encoded_data.drop('ServiceSatisfaction', axis=1)
y = encoded_data['ServiceSatisfaction']

In [40]:
# Split the data into training and testing sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [43]:
def test_models(X_train, y_train, X_test, y_test):
    # Define the models to test
    models = {
        # 'LogisticRegression': LogisticRegression(max_iter=1000, multi_class='multinomial'),
        'KNeighborsClassifier': KNeighborsClassifier(),
        'GaussianNB': GaussianNB(),
        'DecisionTreeClassifier': DecisionTreeClassifier(),
        'AdaBoostClassifier': AdaBoostClassifier(),
        'RandomForestClassifier': RandomForestClassifier(),
        'XGBClassifier': XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
    }
    
    # Prepare the output dataframe
    model_results = []

    for name, model in models.items():
        # Train the model
        model.fit(X_train, y_train)
        
        # Predict on the test set
        y_pred = model.predict(X_test)
        
        # Compute ROC AUC
        roc_auc = None
        if hasattr(model, "predict_proba"):
            y_prob = model.predict_proba(X_test)
            roc_auc = roc_auc_score(y_test, y_prob, multi_class='ovr')
        elif hasattr(model, "decision_function"):
            y_scores = model.decision_function(X_test)
            roc_auc = roc_auc_score(y_test, y_scores, multi_class='ovr')

        # Get classification report
        report = classification_report(y_test, y_pred, output_dict=True)
        
        # Aggregate the results
        model_results.append({
            'Model': name,
            'ROC AUC': roc_auc,
            'Accuracy': report['accuracy'],
            'F1 Score': report['weighted avg']['f1-score'],
            'Support': report['macro avg']['support']  # Total support is the sum of support for all classes
        })

    # Convert list of results to dataframe
    results_df = pd.DataFrame(model_results)
    
    return results_df

# Example usage:
results_df = test_models(X_train, y_train, X_test, y_test)
results_df


c:\Users\mogam\Desktop\MScDSA\Dissertation\clm_app\.venv\lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


,Model,ROC AUC,Accuracy,F1 Score,Support
0,KNeighborsClassifier,0.714486,0.555556,0.559705,108.0
1,GaussianNB,0.638369,0.490741,0.465957,108.0
2,DecisionTreeClassifier,0.623552,0.500000,0.508359,108.0
3,AdaBoostClassifier,0.688757,0.564815,0.569215,108.0
4,RandomForestClassifier,0.713339,0.509259,0.514130,108.0
5,XGBClassifier,0.695369,0.527778,0.537050,108.0


In [ ]:
feature_importances = rf_model.feature_importances_

# Create a Series for the feature importances
importances = pd.Series(feature_importances, index=X_train.columns)

# Sort the importances and select the top 10, then reverse the Series for plotting
top_10_importances = importances.sort_values(ascending=False)[:10][::-1]

# Create a bar chart using Plotly
fig = px.bar(top_10_importances, x=top_10_importances.values, y=top_10_importances.index, orientation='h',
             labels={'x': 'Importance', 'index': 'Feature'},
             title='Top 10 Feature Importances in Random Forest Model (Highest to Lowest)')

# Show the plot
fig.show()

Each bar's length corresponds to the importance score of that feature, with longer bars representing a greater influence on the model's predictions. Features related to HIV services, county of facility, marital status, education level, access challenges, self-identity, and income sources are all key factors in this model's decision-making process.

The specific importance values are not visible, but you can see that the most important feature has an importance score just under 0.04, and the least important feature within this top 10 list has a score slightly above 0. This distribution suggests that 'ExpectedHIVServices_ART medicine' has a relatively more substantial impact compared to the other features in the list.

things to include:
- methods of overfitting
- undersampling of class 3
- better cleaning techniques
- other models to explore
- other interpretations to make from the final selected model